In [1]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [2]:
from bigdl.orca import init_orca_context, stop_orca_context
from bigdl.orca.data.pandas import read_csv   # Replace 'from pandas import read_csv'
from bigdl.orca.learn.tf.estimator import Estimator

In [3]:
sc = init_orca_context(cluster_mode="local", cores=8, memory="4g")
# sc = init_orca_context(cluster_mode="yarn", cores=8, num_nodes=4, memory="4g")

Initializing orca context
Current pyspark location is : /root/anaconda3/envs/orca-demo/lib/python3.7/site-packages/pyspark/__init__.py
Start to getOrCreate SparkContext
pyspark_submit_args is:  --driver-class-path /root/anaconda3/envs/orca-demo/lib/python3.7/site-packages/bigdl/share/dllib/lib/bigdl-dllib-spark_3.1.2-2.0.0-jar-with-dependencies.jar:/root/anaconda3/envs/orca-demo/lib/python3.7/site-packages/bigdl/share/orca/lib/bigdl-orca-spark_3.1.2-2.0.0-jar-with-dependencies.jar pyspark-shell 
2022-05-06 10:21:20 WARN  NativeCodeLoader:60 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


2022-05-06 10:21:22,794 Thread-4 WARN The bufferSize is set to 4000 but bufferedIo is false: false
2022-05-06 10:21:22,796 Thread-4 WARN The bufferSize is set to 4000 but bufferedIo is false: false
2022-05-06 10:21:22,796 Thread-4 WARN The bufferSize is set to 4000 but bufferedIo is false: false
2022-05-06 10:21:22,797 Thread-4 WARN The bufferSize is set to 4000 but bufferedIo is false: false
22-05-06 10:21:22 [Thread-4] INFO  Engine$:121 - Auto detect executor number and executor cores number
22-05-06 10:21:22 [Thread-4] INFO  Engine$:123 - Executor number is 1 and executor cores number is 8
22-05-06 10:21:23 [Thread-4] INFO  ThreadPool$:95 - Set mkl threads to 1 on thread 27
2022-05-06 10:21:23 WARN  SparkContext:69 - Using an existing SparkContext; some configuration may not take effect.
22-05-06 10:21:23 [Thread-4] INFO  Engine$:446 - Find existing spark context. Checking the spark conf...
cls.getname: com.intel.analytics.bigdl.dllib.utils.python.api.Sample
BigDLBasePickler registe


User settings:

   KMP_AFFINITY=granularity=fine,compact,1,0
   KMP_BLOCKTIME=0
   KMP_DUPLICATE_LIB_OK=True
   KMP_INIT_AT_FORK=FALSE
   KMP_SETTINGS=1
   OMP_NUM_THREADS=1

Effective settings:

   KMP_ABORT_DELAY=0
   KMP_ADAPTIVE_LOCK_PROPS='1,1024'
   KMP_ALIGN_ALLOC=64
   KMP_ALL_THREADPRIVATE=224
   KMP_ATOMIC_MODE=2
   KMP_BLOCKTIME=0
   KMP_CPUINFO_FILE: value is not defined
   KMP_DETERMINISTIC_REDUCTION=false
   KMP_DEVICE_THREAD_LIMIT=2147483647
   KMP_DISP_HAND_THREAD=false
   KMP_DISP_NUM_BUFFERS=7
   KMP_DUPLICATE_LIB_OK=true
   KMP_FORCE_REDUCTION: value is not defined
   KMP_FOREIGN_THREADS_THREADPRIVATE=true
   KMP_FORKJOIN_BARRIER='2,2'
   KMP_FORKJOIN_BARRIER_PATTERN='hyper,hyper'
   KMP_FORKJOIN_FRAMES=true
   KMP_FORKJOIN_FRAMES_MODE=3
   KMP_GTID_MODE=3
   KMP_HANDLE_SIGNALS=false
   KMP_HOT_TEAMS_MAX_LEVEL=1
   KMP_HOT_TEAMS_MODE=0
   KMP_INIT_AT_FORK=true
   KMP_ITT_PREPARE_DELAY=0
   KMP_LIBRARY=throughput
   KMP_LOCK_KIND=queuing
   KMP_MALLOC_POOL_INCR=1M
  

In [4]:
full_data = read_csv("ml-1m/ratings.dat", sep="::", header=None, names=["user", "item", "label"],
                     usecols=[0, 1, 2], dtype={0: np.int32, 1: np.int32, 2: np.int32})

2022-05-06 10:21:23,351 Thread-4 WARN The bufferSize is set to 4000 but bufferedIo is false: false
2022-05-06 10:21:23,353 Thread-4 WARN The bufferSize is set to 4000 but bufferedIo is false: false
2022-05-06 10:21:23,356 Thread-4 WARN The bufferSize is set to 4000 but bufferedIo is false: false
2022-05-06 10:21:23,358 Thread-4 WARN The bufferSize is set to 4000 but bufferedIo is false: false
22-05-06 10:21:23 [Thread-4] INFO  Engine$:121 - Auto detect executor number and executor cores number
22-05-06 10:21:23 [Thread-4] INFO  Engine$:123 - Executor number is 1 and executor cores number is 8
22-05-06 10:21:23 [Thread-4] INFO  ThreadPool$:95 - Set mkl threads to 1 on thread 27
2022-05-06 10:21:23 WARN  SparkContext:69 - Using an existing SparkContext; some configuration may not take effect.
22-05-06 10:21:23 [Thread-4] INFO  Engine$:446 - Find existing spark context. Checking the spark conf...


In [5]:
user_set = set(full_data["user"].unique())
item_set = set(full_data["item"].unique())
min_user_id = min(user_set)
max_user_id = max(user_set)
min_item_id = min(item_set)
max_item_id = max(item_set)

min_user_id, max_user_id, min_item_id, max_item_id

(1, 6040, 1, 3952)

In [6]:
def split_train_test(data):
    train, test = train_test_split(data, test_size=0.2, random_state=100)
    return train, test

train_data, test_data = full_data.transform_shard(split_train_test).split()

In [7]:
num_users = max_user_id + 1
num_items = max_item_id + 1

user_input = tf.keras.layers.Input(shape=(1,), dtype="int32")
item_input = tf.keras.layers.Input(shape=(1,), dtype="int32")

# Multi-layer Perceptron
mlp_embed_user = tf.keras.layers.Embedding(input_dim=num_users,
                                           output_dim=10,
                                           input_length=1)(user_input)
mlp_embed_item = tf.keras.layers.Embedding(input_dim=num_items,
                                           output_dim=10,
                                           input_length=1)(item_input)
user_latent = tf.keras.layers.Flatten()(mlp_embed_user)
item_latent = tf.keras.layers.Flatten()(mlp_embed_item)

mlp_latent = tf.keras.layers.concatenate([user_latent, item_latent], axis=1)
mlp_latent = tf.keras.layers.Dense(20, activation="relu")(mlp_latent)
mlp_latent = tf.keras.layers.Dense(10, activation="relu")(mlp_latent)

# Matrix Factorization
mf_embed_user = tf.keras.layers.Embedding(input_dim=num_users,
                                          output_dim=20,
                                          input_length=1)(user_input)
mf_embed_item = tf.keras.layers.Embedding(input_dim=num_items,
                                          output_dim=20,
                                          input_length=1)(item_input)
mf_user_flatten = tf.keras.layers.Flatten()(mf_embed_user)
mf_item_flatten = tf.keras.layers.Flatten()(mf_embed_item)

mf_latent = tf.keras.layers.multiply([mf_user_flatten, mf_item_flatten])
concated_model = tf.keras.layers.concatenate([mlp_latent, mf_latent], axis=1)
prediction = tf.keras.layers.Dense(1, activation="relu")(concated_model)

model = tf.keras.Model([user_input, item_input], prediction)
model.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 1, 10)        60410       input_1[0][0]                    
_____________________________________________________________________________________________

In [8]:
optimizer = tf.keras.optimizers.Adam(1e-2)
model.compile(optimizer=optimizer, loss="mean_squared_error")

estimator = Estimator.from_keras(model)
estimator.fit(train_data,
              batch_size=800,
              epochs=2,
              feature_cols=["user", "item"],
              label_cols=["label"],
              validation_data=test_data)






creating: createFakeOptimMethod
Instructions for updating:
Use `tf.cast` instead.




2022-05-06 10:21:33.952511: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-05-06 10:21:33.952541: E tensorflow/stream_executor/cuda/cuda_driver.cc:318] failed call to cuInit: UNKNOWN ERROR (303)
2022-05-06 10:21:33.952559: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (aep-001): /proc/driver/nvidia/version does not exist
2022-05-06 10:21:33.953365: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 AVX512F FMA
2022-05-06 10:21:33.962844: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2700000000 Hz
2022-05-06 10:21:33.969277: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x555e20407d90 initialized for platform Host (this does not guarantee 




creating: createTFTrainingHelper
linux-x86_64/libiomp5.so
/root/anaconda3/envs/orca-demo/lib/python3.7/site-packages/bigdl/share/orca/lib/../../tflibs/linux-x86_64/libiomp5.so
got shared path
linux-x86_64/libmklml_intel.so
/root/anaconda3/envs/orca-demo/lib/python3.7/site-packages/bigdl/share/orca/lib/../../tflibs/linux-x86_64/libmklml_intel.so
got shared path
linux-x86_64/libtensorflow_framework-zoo.so
/root/anaconda3/envs/orca-demo/lib/python3.7/site-packages/bigdl/share/orca/lib/../../tflibs/linux-x86_64/libtensorflow_framework-zoo.so
got shared path
linux-x86_64/libtensorflow_jni.so
/root/anaconda3/envs/orca-demo/lib/python3.7/site-packages/bigdl/share/orca/lib/../../tflibs/linux-x86_64/libtensorflow_jni.so
got shared path


2022-05-06 10:21:34.586600: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  AVX2 AVX512F FMA
To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-06 10:21:34.590300: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2700000000 Hz
2022-05-06 10:21:34.590564: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fbe930b7f20 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2022-05-06 10:21:34.590585: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version


creating: createIdentityCriterion
creating: createTFParkSampleToMiniBatch
creating: createTFParkSampleToMiniBatch
creating: createEstimator
creating: createMaxEpoch
creating: createEveryEpoch
22-05-06 10:21:34 [Thread-4] INFO  InternalDistriOptimizer$:987 - TFTrainingHelperV2[TFParkTraining] isTorch is false
22-05-06 10:21:34 [Thread-4] INFO  DistriOptimizer$:826 - caching training rdd ...


...

22-05-06 10:23:33 [Thread-4] INFO  DistriOptimizer$:433 - [Epoch 2 674400/800165][Iteration 1844][Wall Clock 107.958654578s] Trained 800.0 records in 0.052447907 seconds. Throughput is 15253.23 records/second. Loss is 0.73180515. 
22-05-06 10:23:33 [Thread-4] INFO  DistriOptimizer$:433 - [Epoch 2 675200/800165][Iteration 1845][Wall Clock 108.013090035s] Trained 800.0 records in 0.054435457 seconds. Throughput is 14696.304 records/second. Loss is 0.5213381. 
22-05-06 10:23:33 [Thread-4] INFO  DistriOptimizer$:433 - [Epoch 2 676000/800165][Iteration 1846][Wall Clock 108.065990033s] Trained 800.0 records in 0.052899998 seconds. Throughput is 15122.874 records/second. Loss is 0.69158876. 
22-05-06 10:23:33 [Thread-4] INFO  DistriOptimizer$:433 - [Epoch 2 676800/800165][Iteration 1847][Wall Clock 108.120804982s] Trained 800.0 records in 0.054814949 seconds. Throughput is 14594.559 records/second. Loss is 0.53089345. 
22-05-06 10:23:33 [Thread-4] INFO  DistriOptimizer$:433 - [Epoch 2 677600/

22-05-06 10:23:35 [Thread-4] INFO  DistriOptimizer$:433 - [Epoch 2 703200/800165][Iteration 1880][Wall Clock 109.9120106s] Trained 800.0 records in 0.054736298 seconds. Throughput is 14615.53 records/second. Loss is 0.52548426. 
22-05-06 10:23:35 [Thread-4] INFO  DistriOptimizer$:433 - [Epoch 2 704000/800165][Iteration 1881][Wall Clock 109.965185744s] Trained 800.0 records in 0.053175144 seconds. Throughput is 15044.623 records/second. Loss is 0.41113847. 
22-05-06 10:23:35 [Thread-4] INFO  DistriOptimizer$:433 - [Epoch 2 704800/800165][Iteration 1882][Wall Clock 110.017485572s] Trained 800.0 records in 0.052299828 seconds. Throughput is 15296.418 records/second. Loss is 0.69485825. 
22-05-06 10:23:35 [Thread-4] INFO  DistriOptimizer$:433 - [Epoch 2 705600/800165][Iteration 1883][Wall Clock 110.06987424s] Trained 800.0 records in 0.052388668 seconds. Throughput is 15270.478 records/second. Loss is 0.49840057. 
22-05-06 10:23:35 [Thread-4] INFO  DistriOptimizer$:433 - [Epoch 2 706400/80

22-05-06 10:23:37 [Thread-4] INFO  DistriOptimizer$:433 - [Epoch 2 732000/800165][Iteration 1916][Wall Clock 111.822581329s] Trained 800.0 records in 0.054012138 seconds. Throughput is 14811.485 records/second. Loss is 0.5223557. 
22-05-06 10:23:37 [Thread-4] INFO  DistriOptimizer$:433 - [Epoch 2 732800/800165][Iteration 1917][Wall Clock 111.87625724s] Trained 800.0 records in 0.053675911 seconds. Throughput is 14904.265 records/second. Loss is 0.4418291. 
22-05-06 10:23:37 [Thread-4] INFO  DistriOptimizer$:433 - [Epoch 2 733600/800165][Iteration 1918][Wall Clock 111.929028337s] Trained 800.0 records in 0.052771097 seconds. Throughput is 15159.814 records/second. Loss is 0.5958795. 
22-05-06 10:23:37 [Thread-4] INFO  DistriOptimizer$:433 - [Epoch 2 734400/800165][Iteration 1919][Wall Clock 111.983925579s] Trained 800.0 records in 0.054897242 seconds. Throughput is 14572.682 records/second. Loss is 0.5213184. 
22-05-06 10:23:37 [Thread-4] INFO  DistriOptimizer$:433 - [Epoch 2 735200/800

22-05-06 10:23:39 [Thread-4] INFO  DistriOptimizer$:433 - [Epoch 2 760800/800165][Iteration 1952][Wall Clock 113.759459082s] Trained 800.0 records in 0.052590973 seconds. Throughput is 15211.736 records/second. Loss is 0.70556074. 
22-05-06 10:23:39 [Thread-4] INFO  DistriOptimizer$:433 - [Epoch 2 761600/800165][Iteration 1953][Wall Clock 113.813947094s] Trained 800.0 records in 0.054488012 seconds. Throughput is 14682.129 records/second. Loss is 0.73271143. 
22-05-06 10:23:39 [Thread-4] INFO  DistriOptimizer$:433 - [Epoch 2 762400/800165][Iteration 1954][Wall Clock 113.866926945s] Trained 800.0 records in 0.052979851 seconds. Throughput is 15100.081 records/second. Loss is 0.60444623. 
22-05-06 10:23:39 [Thread-4] INFO  DistriOptimizer$:433 - [Epoch 2 763200/800165][Iteration 1955][Wall Clock 113.919178221s] Trained 800.0 records in 0.052251276 seconds. Throughput is 15310.631 records/second. Loss is 0.60162276. 
22-05-06 10:23:39 [Thread-4] INFO  DistriOptimizer$:433 - [Epoch 2 76400

22-05-06 10:23:41 [Thread-4] INFO  DistriOptimizer$:433 - [Epoch 2 789600/800165][Iteration 1988][Wall Clock 115.702944604s] Trained 800.0 records in 0.05338356 seconds. Throughput is 14985.888 records/second. Loss is 0.6559123. 
22-05-06 10:23:41 [Thread-4] INFO  DistriOptimizer$:433 - [Epoch 2 790400/800165][Iteration 1989][Wall Clock 115.75826895s] Trained 800.0 records in 0.055324346 seconds. Throughput is 14460.18 records/second. Loss is 0.7847697. 
22-05-06 10:23:41 [Thread-4] INFO  DistriOptimizer$:433 - [Epoch 2 791200/800165][Iteration 1990][Wall Clock 115.811136147s] Trained 800.0 records in 0.052867197 seconds. Throughput is 15132.257 records/second. Loss is 0.5259159. 
22-05-06 10:23:41 [Thread-4] INFO  DistriOptimizer$:433 - [Epoch 2 792000/800165][Iteration 1991][Wall Clock 115.864058665s] Trained 800.0 records in 0.052922518 seconds. Throughput is 15116.439 records/second. Loss is 0.6905806. 
22-05-06 10:23:41 [Thread-4] INFO  DistriOptimizer$:433 - [Epoch 2 792800/80016

In [9]:
model = estimator.get_model()
tf.saved_model.save(model, "./model")

2022-05-06 10:23:43.938419: W tensorflow/python/util/util.cc:299] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: ./model/assets


In [10]:
stop_orca_context()

Stopping orca context
